### import

In [23]:
import pandas as pd
import numpy as np
from os.path import join
from datetime import datetime
import functions
import warnings
warnings.simplefilter("ignore")
deposit_dtype = functions.deposit_dtype

In [ ]:
# ################################
path_deposit = r"D:\3.자산\전산 dataset\솔림\기간축적데이터\입금"
fn_write = "솔림입금전체_160119~250630.pkl"
# ################################

# # 입금조회새창들 병합
# fnlist = functions.키워드로파일명찾기(path_deposit, 포함키워드="입금조회새창", 여러파일허용=True)
# if isinstance(fnlist, str) : # 파일 하나인 경우
#     deposit = pd.read_excel(fnlist, dtype=deposit_dtype)
# else : # 파일 여러개(리스트)인 경우
#     deposit = pd.DataFrame(None)
#     for fn in fnlist :
#         deposit = pd.concat([deposit, pd.read_excel(fn, dtype=deposit_dtype)])
        
# # PKL 저장
# deposit.to_pickle(join(path_deposit, fn_write))

In [26]:
deposit = pd.read_pickle(functions.키워드로파일명찾기(path_deposit, 포함키워드="입금전체"))

In [ ]:
종결일 = pd.read_excel(r"D:\3.자산\전산 dataset\솔림\기간축적데이터\입금\계좌조회새창_20250717_1702.xlsx")
종결일["종결일"] = pd.to_datetime(종결일.종결일)
# 종결일 = 종결일[["계좌키","종결일"]].dropna(subset=['종결일'], axis=0)

In [56]:
종결일["계좌키"] = 종결일["계좌키"].astype(str)

In [32]:
deposit.columns

Index(['채무자키', '채권구분', '매각사구분', '입금키', '계좌키', '고객입금일', '입금일', '입금합계', '채무상태',
       '담당자', '입금원금', '입금법비용', '입금미수이자', '입금연체이자', '입금정상이자', '연체일수', '적용이자율',
       '입금기타', '입금자', '입금자구분', '입금구분', '담당입금구분', '입금통계구분', '입금기간', '입금기간일',
       '입금은행', '입금계좌번호', '가상계좌은행', '가상계좌번호', '상대방입금은행', '입금메모', '입금비고', '계좌번호',
       '회수원금', '회수법비용', '회수미수이자', '회수연체이자', '회수기타', '회수합계', '현재원금', '현재법비용',
       '현재미수이자', '현재연체이자', '현재합계', '원가입금', '매출입금', '비용입금', '잡이익', '법조치키',
       '입금고정키', '등록파일', '등록일자', '등록유저', '수정일자', '수정유저', '입금순서', '취소여부', '종결일',
       '매각사', '담보제공처', '담보제공구분', '매입일', '컷오프일', '최초이자기준일', '주민번호인', '지점', '부서',
       '팀', '고객입금액', '채무자명', '타입금키', '타채무자키', '보증인키', '직접입력메모', '담당문자전송',
       '리파이낸싱', '입금수수료율', '매입사'],
      dtype='object')

In [35]:
deposit["입금일"] = pd.to_datetime(deposit["입금일"])

In [57]:
merged = pd.merge(deposit[["채무자키","계좌키","입금키","입금구분","입금일","입금기간","담당자","입금합계"]], 종결일, how='inner')

In [58]:
merged["종결후입금"] = np.where(merged.입금일 <= merged.종결일, "종결전", "종결후")

In [59]:
rst = merged[(merged["종결후입금"] == "종결후") & (merged["입금기간"] != "매각후") & (merged.입금구분!="법원환급금")]

In [60]:
rst.to_excel(join(path_deposit, "매각후입금.xlsx"), index=False)

In [62]:
deposit[(deposit.입금기간=="원상회복전") & (deposit.입금구분!="신용회복")][["채무자키","계좌키","입금키","입금일","입금자구분","입금구분","입금합계","담당자","채무상태"]].to_excel('원상회복 확인건.xlsx')